In [1]:
import pandas as pd
import numpy as np

train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
train_data.set_index('PassengerId', inplace=True)
test_data.set_index('PassengerId', inplace=True)
train_data.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
train_data['Embarked'].value_counts()

Embarked
S    644
C    168
Q     77
Name: count, dtype: int64

In [3]:
train_data[['Embarked', 'Pclass', 'Survived']]

,Embarked,Pclass,Survived
PassengerId,,,
1,S,3,0
2,C,1,1
3,S,3,1
4,S,1,1
5,S,3,0
...,...,...,...
887,S,2,0
888,S,1,1
889,S,3,0


In [4]:
train_data.groupby(['Embarked', 'Pclass'])['Survived'].mean()

Embarked  Pclass
C         1         0.694118
          2         0.529412
          3         0.378788
Q         1         0.500000
          2         0.666667
          3         0.375000
S         1         0.582677
          2         0.463415
          3         0.189802
Name: Survived, dtype: float64

In [5]:
# let's make a simple model that literally just assumes all 3rd class peopl die

def simple_lives(seat_class):
    return int(seat_class < 3)



In [6]:
test_data.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [7]:
# simple predictions
simple_predictions = pd.DataFrame({'PassengerId': test_data.index, 'Survived': test_data['Pclass'].map(simple_lives)})
simple_predictions.head()

,PassengerId,Survived
PassengerId,,
892,892,0
893,893,0
894,894,1
895,895,0
896,896,0


In [8]:
simple_predictions.to_csv('simple_submission.csv', index=False)

In [9]:
#score was 0.603, which can definitely be improved on! 
# let's make a model with only numerical features
only_numerical_train = train_data.select_dtypes(include=np.number)
only_numerical_train.head()

,Survived,Pclass,Age,SibSp,Parch,Fare
PassengerId,,,,,,
1,0,3,22.0,1,0,7.2500
2,1,1,38.0,1,0,71.2833
3,1,3,26.0,0,0,7.9250
4,1,1,35.0,1,0,53.1000
5,0,3,35.0,0,0,8.0500


In [10]:
def get_features_target_df(df):
    return df.drop('Survived', axis=1), df['Survived']

In [11]:
X,y = get_features_target_df(only_numerical_train)
y

PassengerId
1      0
2      1
3      1
4      1
5      0
      ..
887    0
888    1
889    0
890    1
891    0
Name: Survived, Length: 891, dtype: int64

In [12]:
#let's do a basic decision tree to be honest

from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(random_state=0)
clf.fit(X,y)

DecisionTreeClassifier(random_state=0)

In [13]:
#decision tree predictions
only_numerical_test = test_data.select_dtypes(include=np.number)
tree_predictions = {'Survived': clf.predict(only_numerical_test), 'PassengerId': only_numerical_test.index}
pd.DataFrame(tree_predictions).to_csv('tree_submission.csv', index=False)


In [17]:
# wow! this did worse at 0.581, maybe there's more information than we think!
# let's try incorporating our other features via one-hot-encoding
features_for_encoding = set(train_data.columns) - set(only_numerical_train.columns)
# I think name is too complicated as well, so let's remove that 
features_for_encoding.remove('Name')
features_for_encoding

encoded_train = pd.get_dummies(train_data.drop('Name', axis=1))
encoded_train.head()


,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Ticket_110152,Ticket_110413,...,Cabin_F G73,Cabin_F2,Cabin_F33,Cabin_F38,Cabin_F4,Cabin_G6,Cabin_T,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,,,,,,,,,,,,,
1,0,3,22.0,1,0,7.2500,False,True,False,False,...,False,False,False,False,False,False,False,False,False,True
2,1,1,38.0,1,0,71.2833,True,False,False,False,...,False,False,False,False,False,False,False,True,False,False
3,1,3,26.0,0,0,7.9250,True,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4,1,1,35.0,1,0,53.1000,True,False,False,False,...,False,False,False,False,False,False,False,False,False,True
5,0,3,35.0,0,0,8.0500,False,True,False,False,...,False,False,False,False,False,False,False,False,False,True


In [18]:
# hmmm seems like a lot of columns (839)
# let's check that the ticket or cabin columns arent unique/significant
print(train_data['Ticket'].value_counts())
print(train_data['Cabin'].value_counts())

# i don't think these say too much either. let's drop them

Ticket
347082      7
CA. 2343    7
1601        7
3101295     6
CA 2144     6
           ..
9234        1
19988       1
2693        1
PC 17612    1
370376      1
Name: count, Length: 681, dtype: int64
Cabin
B96 B98        4
G6             4
C23 C25 C27    4
C22 C26        3
F33            3
              ..
E34            1
C7             1
C54            1
E36            1
C148           1
Name: count, Length: 147, dtype: int64


In [21]:
encoded_train = pd.get_dummies(train_data.drop(['Name', 'Ticket', 'Cabin'], axis=1))
encoded_train.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,,,
1,0,3,22.0,1,0,7.2500,False,True,False,False,True
2,1,1,38.0,1,0,71.2833,True,False,True,False,False
3,1,3,26.0,0,0,7.9250,True,False,False,False,True
4,1,1,35.0,1,0,53.1000,True,False,False,False,True
5,0,3,35.0,0,0,8.0500,False,True,False,False,True


In [22]:
#let's reuse the same model
X,y = get_features_target_df(encoded_train)

clf = DecisionTreeClassifier(random_state=0)
clf.fit(X,y)

DecisionTreeClassifier(random_state=0)

In [23]:
# let's make our predictions and send them in
encoded_test = pd.get_dummies(test_data.drop(['Name', 'Ticket', 'Cabin'], axis=1))
tree_predictions = {'Survived': clf.predict(encoded_test), 'PassengerId': encoded_test.index}
pd.DataFrame(tree_predictions).to_csv('better_tree_submission.csv', index=False)

In [ ]:
# nice! a score of 0.734